In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

# from keras.utils.np_utils import to_categorical
import sklearn
from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_csv('data/data.csv', sep=';')
# sort data by team and year
df = df.sort_values(['team', 'year'], ascending=[True, True]).reset_index(drop=True)
df.tail()

In [ ]:
df_dummies = pd.get_dummies(df, columns=['team'])
df_dummies.head()


In [ ]:
df_filtered = df_dummies.drop(columns=['year'])
df_filtered.head()

In [ ]:
vals = df.values
sequences = 2
teams = np.unique(df['team'].values)
print(teams)
X = []
y = []
X_test = []
X_test_teams = []

df_dummies = pd.get_dummies(df, columns=['team'])
df_filtered = df_dummies.drop(columns=['year'])

for team in teams:
    df_team = df_filtered.loc[df['team'] == team]
    #print(df_team)
    team_standings = df_team.values
    # print(f"len(team_standings) {len(team_standings)}")
    for i in range(0, len(team_standings)):
        # print(f"team {team} {i}th year")
        if i + sequences < len(team_standings):
            x_values = team_standings[i:i+sequences]
            # print(x_values)
            # X.append(x_values)
            y_value = team_standings[i+sequences, 0]
            #print(y_value)
            y.append(y_value)
        elif i + sequences == len(team_standings):
            x_values = team_standings[i:i+sequences]
            X_test.append(x_values)
            print(f"team {team}")
            X_test_teams.append(team)

X = np.array(x_values)
y = np.array(y)
X_test = np.array(X_test)
print(X.shape)
print(y.shape)
print(X_test.shape)

In [ ]:
print(y)

In [ ]:
print(X_test)

In [ ]:
def linear_transform(X):
    if len(X.shape) == 3:
        if X.shape[1] == 1:
            X_linear = np.squeeze(X, 1)
        else:
            X_linear = X.reshape((X.shape[0], X.shape[1]* X.shape[2]))
    return X_linear

In [ ]:
X_linear = linear_transform(X)
print(X_linear.shape)

In [ ]:
print(X_linear.shape)
print(y.shape)
reg = LinearRegression().fit(X_linear, y)
reg.score(X_linear, y)

In [ ]:
X_test_linear = linear_transform(X_test)
print(X_test_linear.shape)

In [ ]:
print(X_test_teams)

In [ ]:
result = reg.predict(X_test_linear)
print(np.array(result))

In [ ]:
inds = np.argsort(result, axis=0)
print(inds)

In [ ]:
for i, ind in enumerate(inds):
    team = X_test_teams[ind]
    res = result[ind]
    print(f"{i+1}th team {team} is {round(res, 2)}")

# LSTM

In [ ]:
def set_dataset(X, y):
    train_ds = tf.data.Dataset.from_tensor_slices((X, y))
    return train_ds

def preprocessing(df, sequences):
    vals = df.values
    teams = np.unique(df['team'].values)
    print(teams)
    X = []
    y = []
    X_test = []
    X_test_teams = []

    # df_dummies = pd.get_dummies(df, columns=['team'])
    df_filtered = df.drop(columns=['year'])

    for team in teams:
        df_team = df_filtered.loc[df['team'] == team]
        # print(df_team)
        team_standings = df_team.standing.values
        print(f"team_standings {team_standings}")
        for i in range(0, len(team_standings)):
            print(f"team {team} {i}th year")
            if i + sequences < len(team_standings):
                x_values = team_standings[i:i+sequences]
                print(f"x_values {x_values}")
                X.append(x_values)
                y_value = team_standings[i+sequences]
                print(f"y_value {y_value}")
                y.append(y_value)
            elif i + sequences == len(team_standings):
                x_values = team_standings[i:i+sequences]
                X_test.append(x_values)
                print(f"team {team}")
                X_test_teams.append(team)

    X = np.array(X)
    X = X.reshape(X.shape[0], X.shape[1], 1)
    y = np.array(y)
    X_test = np.array(X_test)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
    print(X.shape)
    print(y.shape)
    print(X_test.shape)
    return X, y, X_test, X_test_teams

In [ ]:
def set_lstm(n_timesteps, n_features, hidden, n_outputs):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(hidden, input_shape=(n_timesteps, n_features)))
    model.add(tf.keras.layers.Dropout(0.1))
    #model.add(Dense(hidden, activation='relu'))
    model.add(tf.keras.layers.Dense(n_outputs, activation='relu'))
    # model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['mse'])
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    model.summary()
    return model

def predict(model, X_test):
    lstm_preds = model.predict(X_test)
    inds = np.argsort(lstm_preds[:,0])
    print(inds)
    print(lstm_preds.shape)

    for i, ind in enumerate(inds):
        team = X_test_teams[ind]
        pred = lstm_preds[ind, 0]
        print(f"{i+1}th team {team} is {str(np.round(pred, 1))}")

def train(n_timesteps, n_features, hidden, output_size, batch_size, epochs):
    model = set_lstm(n_timesteps, n_features, hidden, output_size)
    # Compile the model
    # optimizer = tf.keras.optimizers.Adam()
    # model.compile(optimizer=optimizer, loss='mse')

    # train_ds = set_dataset(X, y)
    print(X.shape)
    X_train, X_val = X[:100,:,:], X[100:,:]
    y_train, y_val = y[:100], y[100:]

    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=2, validation_split=0.2)
    # evaluate model
    _, accuracy = model.evaluate(X_val, y_val, batch_size=batch_size, verbose=0)

    print(accuracy)
    return model

In [ ]:
df = pd.read_csv('data/data.csv', sep=';')
# sort data by team and year
df = df.sort_values(['team', 'year'], ascending=[True, True]).reset_index(drop=True)
df.tail()

In [ ]:
n_timesteps = 3
n_features = 1
hidden = 64
output_size = 1  # labels are from 1-21
batch_size = 4
epochs = 50

X, y, X_test, X_test_teams = preprocessing(df, n_timesteps)

model = train(n_timesteps, n_features, hidden, output_size, batch_size, epochs)


In [ ]:
predict(model, X_test)